In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import random_split

In [3]:
from torchvision import transforms, datasets
train_set = "/kaggle/input/inature12k/inaturalist_12K/train"
test_set = "/kaggle/input/inature12k/inaturalist_12K/val"
IMG_SIZE = (224,224)
# Transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),  # Resize the images
    transforms.ToTensor(),        
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) ])  # Normalize the images


# Use ImageFolder to create a dataset
trainset = datasets.ImageFolder(root=train_set, transform=transform)
val_size = int(0.2 * len(trainset))
train_size = len(trainset) - val_size
trainset, valset = random_split(trainset, [train_size, val_size])
testset = datasets.ImageFolder(root=test_set, transform=transform)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class pretrainedCNN():
    def __init__(self, trainset, valset, model, batch_size=32, freeze_percent=1):
        self.model = model

        # Freeze layers based on freeze_percent
        layers = list(self.model.parameters())
        num_layers_to_freeze = int(freeze_percent * len(layers))

        for i, param in enumerate(layers):
            param.requires_grad = False if i < num_layers_to_freeze else True

        # Replace the final FC layer
        num_classes = 10  
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)
        self.model.fc.to(device)

      
        self.dataloader_train = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
        self.dataloader_val = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

    def train(self, epochs=10, lr=0.001, weight_decay=0):
        self.model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(filter(lambda p: p.requires_grad, self.model.parameters()),
                               lr=lr, weight_decay=weight_decay)

        for epoch in range(1, epochs + 1):
            self.model.train()
            train_accuracy, train_loss = 0, 0
            total_train = 0

            for inputs, labels in self.dataloader_train:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * inputs.size(0)
                train_accuracy += (outputs.argmax(1) == labels).sum().item()
                total_train += labels.size(0)

            train_acc = train_accuracy / total_train

            # Validation
            self.model.eval()
            val_accuracy, val_loss = 0, 0
            total_val = 0

            with torch.no_grad():
                for inputs, labels in self.dataloader_val:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = self.model(inputs)
                    loss = criterion(outputs, labels)

                    val_loss += loss.item() * inputs.size(0)
                    val_accuracy += (outputs.argmax(1) == labels).sum().item()
                    total_val += labels.size(0)

            val_acc = val_accuracy / total_val
            avg_val_loss = val_loss / total_val

            print(f"Epoch {epoch}: Train Acc: {train_acc*100:.2f}%  Val Acc: {val_acc*100:.2f}%")
     




In [7]:
# Load pretrained model (e.g., ResNet18)
model = models.resnet18(pretrained=True)

# Initialize your training class with best hyperparameters
trainer = pretrainedCNN(
    trainset=trainset,
    valset=valset,
    model=model,
    batch_size=64,
    freeze_percent=0.9
)

# Train the model
trainer.train(epochs=10, lr=1e-4, weight_decay=0.0005)

# Create a test dataloader
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Evaluate on test set
def evaluate_test_accuracy(model, testloader):
    model.eval()
    model.to(device)
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total * 100
    print(f"\n Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Run test evaluation
test_accuracy = evaluate_test_accuracy(trainer.model, testloader)


KeyboardInterrupt: 